# Compare different Results from different servers

1. This notebook will target the directories `coffeelake1`, `alder`, and `bombay`

2. The files needed inside each directory are `All_Optimizations-<hostanme>.csv` and `All_OptimizationsSYCL-CUDA-<hostname>.csv`

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from common.utils import get_best_optimization

# list of hosts
host_list = ['coffeelake1', 'alder', 'bombay']

tbb_res = {}
sycl_res = {}

# get all the results
for hostname in host_list:
    # get the TBB results
    df = pd.read_csv(os.path.join(hostname, f'All_Optimizations-{hostname}.csv'), sep=';')
    # get Total times
    df.insert(4,"Total",0)
    df['Total']=df['TimeTree']+df['TimeOWM']
    # get the best optimization
    _,besttbb_label = get_best_optimization(df)
    print(f'Best optimization for TBB in {hostname}: {besttbb_label}')
    # keep only Baseline and the best optimization
    df = df.loc[df['Optimization'].isin(['Baseline', besttbb_label])]
    df.reset_index(drop=True, inplace=True)
    # keep the dataframe
    tbb_res[hostname] = df.copy()

    # get SYCL results
    dfs=pd.read_csv(os.path.join(hostname, f'All_OptimizationsSYCL-CUDA-{hostname}.csv'), sep=';')
    # get the Total times
    dfs.insert(4,"Total",0)
    dfs['Total']=dfs['TimeTree']+dfs['TimeOWM']
    # keep only memo versions
    dfm = dfs[~dfs["Optimization"].str.contains("nomemo")].copy()
    # keep the best version between CUDA and CUDA-grid
    opts = dfm['Optimization'].unique()
    if 'owm-cuda' in opts and 'owm-cuda-grid' in opts:
        if dfm.loc[dfm['Optimization'] == 'owm-cuda', 'Total'].mean() > dfm.loc[dfm['Optimization'] == 'owm-cuda-grid', 'Total'].mean():
            # drop CUDA without grid
            drop_label = 'owm-cuda'
        else:
            drop_label = 'owm-cuda-grid'
        # drop the selected label
        dfm = dfm[~dfm['Optimization'].isin([drop_label])].copy()
    # keep the dataframe
    sycl_res[hostname] = dfm.copy()


# Compare each implementation against the others hosts

In [ ]:
# supose coffeelake1 as the reference
tbb_ref,_ = get_best_optimization(tbb_res['coffeelake1'])

for hostname in tbb_res:
    if hostname == 'coffeelake1':
        continue
    # get the best optimization
    besttbb,besttbb_label = get_best_optimization(tbb_res[hostname])
    ref = np.array(tbb_ref)
    best = np.array(besttbb)
    speedupTBB = ref/best
    print(f"Speedup O4 {hostname} against coffeelake1:")
    print(speedupTBB)
